# Исследование статистики чемпионата мира по автогонкам класса Формулы-1 с 1950 по 2017 год

In [133]:
from pandas import Series, DataFrame, read_csv
import pandas as pd
import numpy as np
import datetime

## Для начала посчитаем общую статистику

In [140]:
seasons = read_csv('dataset/f1_seasons.csv')
races = read_csv('dataset/f1_races.csv')
races = races.loc[(races['date'] < datetime.datetime.today().strftime('%Y-%m-%d'))]
print 'Общее количество проведенных чемпионатов: {}'.format(seasons.shape[0])
print 'Общее количество проведенных гонок: {}'.format(races.shape[0])

Общее количество проведенных чемпионатов: 68
Общее количество проведенных гонок: 970


## А теперь изучим географию

### Сколько всего стран принимали этапы чемпионата мира

In [141]:
circuits = read_csv('dataset/f1_circuits.csv')
races = races.merge(circuits, left_on='circuit', right_on='name', how='left')
races = races[['year', 'country', 'location']]
print 'Итого: {}'.format(races.country.unique().shape[0])

Итого: 32


### Сколько всего городов принимали этапы чемпионата мира

In [142]:
print 'Итого: {}'.format(races.location.unique().shape[0])

Итого: 69


### Сделаем группировку по количеству Гранд-при в каждой стране

In [143]:
races.groupby('country')['country'] \
.agg(['count']) \
.sort_values('count', ascending=False)

,count
country,
Italy,95
Germany,76
UK,71
USA,67
Monaco,64
Belgium,62
France,59
Spain,54
Canada,48


Как видно, больше всего Ф-1 любит приезжать в Италию, а в Марокко, видимо, заехали по случайности.

### Сделаем группировку по наиболее популярным городам Гранд-при

In [144]:
cities = races.groupby(['location', 'country'])['location'] \
.agg(['count']) \
.sort_values('count', ascending=False)
cities.head(10)

,,count
location,country,
Monza,Italy,67
Monte-Carlo,Monaco,64
Silverstone,UK,51
Spa,Belgium,50
Nürburg,Germany,40
Montreal,Canada,38
Hockenheim,Germany,35
São Paulo,Brazil,34
Budapest,Hungary,32


Как видно, ни один город не принимал этапы все 68 сезонов подряд, но итальянский город Монца был к этому очень близок.

### Для наглядности, 10-ка наименее популярных городов

In [145]:
cities.tail(10)

,,count
location,country,
Baku,Azerbaijan,2
Castle Donington,UK,1
Dallas,USA,1
Le Mans,France,1
Styria,Austria,1
Casablanca,Morocco,1
Lisbon,Portugal,1
Florida,USA,1
Pescara,Italy,1


## С географией более-менее разобрались, можно переходить к гонщикам

In [149]:
drivers = read_csv('dataset/f1_drivers.csv')
results = read_csv('dataset/f1_results.csv')

### Всего гонщиков хотя бы раз выходило на старт Гранд-при

In [150]:
print 'Итого: {}'.format(drivers.shape[0])

Итого: 840


### Общее количество стран, которые представляли гонщики

In [151]:
print 'Итого: {}'.format(drivers.nationality.unique().shape[0])

Итого: 41


### Посмотрим статистику распределения гонщиков по странам

In [152]:
drivers.groupby('nationality')['nationality'] \
.agg(['count']) \
.sort_values('count', ascending=False)

,count
nationality,
British,162
American,157
Italian,99
French,72
German,49
Brazilian,31
Argentine,24
South African,23
Swiss,23


С большим отрывом лидируют британцы и американцы, но такое количество американцев можно объяснить наличием гонки Инди 500, которая формально входила в чемпионат в 50-е годы, проверим это предположение:

In [199]:
drivers_results = results.merge(drivers, on='driverRef', how='left')
drivers_results_50 = drivers_results[['nationality', 'driverRef']] \
.loc[(drivers_results['year'] >= 1950) & (drivers_results['year'] <= 1960)]
drivers_results_50 = drivers_results_50.drop_duplicates()
drivers_results_50.groupby('nationality')['nationality'] \
.agg(['count']) \
.sort_values('count', ascending=False)

,count
nationality,
American,128
British,82
Italian,29
French,26
German,21
Argentine,16
Belgian,13
Swiss,8
Brazilian,4
